### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB (продолжение)

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd

In [2]:
def _parser_superjob(vacancy):
    vacancy_date = []

    params = {
        'keywords': vacancy,
        'profession_only': '1',
        'geo[c][1]': '1',
        'page': ''
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }

    link = 'https://www.superjob.ru/vacancy/search/'

    req = requests.get(link, params=params, headers=headers)
    soup = bs(req.text, 'html.parser')

    page_block = soup.find('div', {'class': '_3zucV L1p51 undefined _1Fty7 _2tD21 _3SGgo'})

    if not page_block:
        last_page = 1
    else:
        last_page = int(page_block.find_all('a')[-2].getText())

    for page in range(0, last_page):
        params['page'] = page
        req = requests.get(link, params=params, headers=headers)

        parsed_html = bs(req.text, 'html.parser')
        vacancy_items = parsed_html.find_all('div', {'class': 'Fo44F QiY08 LvoDO'})

        for j_vacancy in vacancy_items:
            vacancy_date.append(_parser_item_superjob(j_vacancy))

    return vacancy_date


In [3]:
def _parser_item_superjob(j_vacancy):
    vacancy_data = {}
    vacancy_name = j_vacancy.find('a').getText()
    vacancy_link = 'https://russia.superjob.ru' + j_vacancy.find('a').get('href')
    vacancy_city = j_vacancy.find('span',
                                {'class': '_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'}).getText()
    vacancy_city = re.split(r'\•', vacancy_city)
    vacancy_city = vacancy_city[-1]
    
    try:
        vacancy_employer = j_vacancy.find('span', {
            'class': '_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'}).getText()
    except:
        vacancy_employer = 'Нет названия фирмы'

    try:
        vacancy_salary = j_vacancy.find('span', {'class': '_3mfro _2Wp8I PlM3e _2JVkc _2VHxz'}).getText()

        vacancy_salary = re.split(r'\s|-', vacancy_salary)

        if vacancy_salary[0] == 'до':
            salary_min = None
            salary_max = int((vacancy_salary[1]) + (vacancy_salary[2]))
            salary_currency = vacancy_salary[-1]

        elif vacancy_salary[0] == 'от':
            salary_min = int((vacancy_salary[1]) + (vacancy_salary[2]))
            salary_max = None
            salary_currency = vacancy_salary[-1]

        elif vacancy_salary[0] == 'По':
            salary_min = None
            salary_max = None
            salary_currency = None

        else:
            salary_min = int((vacancy_salary[0]) + (vacancy_salary[1]))
            salary_max = int((vacancy_salary[3]) + (vacancy_salary[4]))
            salary_currency = vacancy_salary[-1]


    except:
        salary_min = None
        salary_max = None
        salary_currency = None

    vacancy_data['name'] = vacancy_name
    vacancy_data['employer'] = vacancy_employer
    vacancy_data['city'] = vacancy_city
    vacancy_data['salary_min'] = salary_min
    vacancy_data['salary_max'] = salary_max
    vacancy_data['salary_currency'] = salary_currency
    vacancy_data['link'] = vacancy_link
    vacancy_data['site'] = 'superjob.ru'

    return vacancy_data

In [4]:
a = _parser_superjob(input('Вы ищите: '))
df = pd.DataFrame(a)
b = int(input('Количество вакансий: '))
df.head(b)

Вы ищите: python
Количество вакансий: 20


,name,employer,city,salary_min,salary_max,salary_currency,link,site
0,Python developer,Синема Стар,"Москва, Выставочная",NaN,NaN,None,https://russia.superjob.ru/vakansii/python-dev...,superjob.ru
1,"Web-программист PHP, Python, ASP.NET Core MVC 2.x",Вычислительный центр,"Нижний Новгород, Горьковская",NaN,NaN,None,https://russia.superjob.ru/vakansii/web-progra...,superjob.ru
2,Senior Python Developer,05.RU,Махачкала,NaN,NaN,None,https://russia.superjob.ru/vakansii/senior-pyt...,superjob.ru
3,Python / Full stack-разработчик,Частный рекрутер Бельтюкова Мария Александровна,Москва,150000.0,180000.0,руб.,https://russia.superjob.ru/vakansii/python-343...,superjob.ru
4,Разработчик Python Middle\Senior,Фидес,Санкт-Петербург,NaN,NaN,None,https://russia.superjob.ru/vakansii/razrabotch...,superjob.ru
5,Разработчик Python,"БСКБ ""Нефтехимавтоматика""",Уфа,NaN,NaN,None,https://russia.superjob.ru/vakansii/razrabotch...,superjob.ru
6,Full-stack-веб-программист python/javascript,ФГБУ Центр стратегического планирования МФБА Р...,Москва,NaN,NaN,None,https://russia.superjob.ru/vakansii/full-stack...,superjob.ru
7,Программист Python,КИБЕРБОРЕЯ,Москва,120000.0,150000.0,руб.,https://russia.superjob.ru/vakansii/programmis...,superjob.ru
8,Senior Python Developer,"РА ""Подбор персонала @ HR аудит""",Москва,NaN,NaN,None,https://russia.superjob.ru/vakansii/senior-pyt...,superjob.ru
9,Ведущий разработчик Python,"БСКБ ""Нефтехимавтоматика""",Уфа,NaN,NaN,None,https://russia.superjob.ru/vakansii/veduschij-...,superjob.ru


### Урок 3. Системы управления базами данных MongoDB и SQLite в Python

#### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, записывающую собранные вакансии в созданную БД

In [5]:
from pymongo import MongoClient
from pprint import pprint

Создаем базу данных в MongoDB и загружаем в нее данные

In [6]:
client = MongoClient('127.0.0.1',27017)
db = client['job_openings']
superjob = db.superjob

df_dict = df.to_dict('records')

superjob.insert_many(df_dict)

#### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введенной суммы. Поиск по двум полям (мин и макс зарплату)

In [7]:
def print_salary(salary):
    objects = superjob.find({'$or': [{'salary_min': {'$gt': salary}}, {'salary_max': {'$gt': salary}}]}, {'_id': 0})
    for obj in objects:
        pprint(obj)

In [8]:
salary = float(input('Уровень зарплаты от: '))
print_salary(salary)

Уровень зарплаты от: 120000
{'city': ' Москва',
 'employer': 'Частный рекрутер Бельтюкова Мария Александровна',
 'link': 'https://russia.superjob.ru/vakansii/python-34337426.html',
 'name': 'Python / Full stack-разработчик',
 'salary_currency': 'руб.',
 'salary_max': 180000.0,
 'salary_min': 150000.0,
 'site': 'superjob.ru'}
{'city': ' Москва',
 'employer': 'КИБЕРБОРЕЯ',
 'link': 'https://russia.superjob.ru/vakansii/programmist-python-34345798.html',
 'name': 'Программист Python',
 'salary_currency': 'руб.',
 'salary_max': 150000.0,
 'salary_min': 120000.0,
 'site': 'superjob.ru'}


#### 3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта

In [9]:
# for i in df_dict:
#     superjob.updata_many({'link':i['link']}, {'$set':i}, upsert=True)

Верхний вариант у меня не работает, выдает ошибку:"TypeError: 'Collection' object is not callable. If you meant to call the 'updata_many' method on a 'Collection' object it is failing because no such method exists.". Я думаю, что у меня как-то коряво установилась MongoDB.

In [10]:
m = 0
n = 0
for i in df_dict:

    if superjob.count_documents({'link': i['link']}) > 0:
        m += 1
    else:
        superjob.insert_one(i)
        n += 1

In [11]:
superjob.delete_many({})